In [1]:
from scapy.all import *
load_contrib('bgp')

In [2]:
base = "/home/greg/BGP_Chain/Seed_scalable_complete/bgp_smart_contracts/pcaps/2022-10-17-14:37:04/"
pcap_path = base + "as150.pcap"
# multi_pcap_path = base + "multiple-nlri.pcap"
hijack_base = "/home/greg/BGP_Chain/Seed_scalable_complete/bgp_smart_contracts/pcaps/"
as151_hijack = hijack_base + "as151-hijack.pcap"

In [3]:
def get_bgp_update_packet():
    for pkt in PcapReader(pcap_path):
        try:
            if (str(pkt.summary()).find('BGPHeader') > 0) and (pkt[BGPHeader].type == 2):
                return pkt
                print(pkt[TCP].show())
    #             print(packet[TCP].payload())
        except Exception as e:
            print(repr(e))
            
def get_bgp_update_with_2_nlri():
    for pkt in PcapReader(multi_pcap_path):
        try:
            if (str(pkt.summary()).find('BGPHeader') > 0) and (pkt[BGPHeader].type == 2):
                if len(pkt[BGPUpdate].nlri) > 1:
                    return pkt
    #             print(packet[TCP].payload())
        except Exception as e:
            print(repr(e))
            
def get_bgp_update_151_hijack_packet():
    count = 0
    for pkt in PcapReader(as151_hijack):
        count += 1
        try:
            if (str(pkt.summary()).find('BGPHeader') > 0) and (pkt[BGPHeader].type == 2):
                if len(pkt[BGPUpdate].nlri) > 1 and count > 27:
                    return pkt
    #             print(packet[TCP].payload())
        except Exception as e:
            print(repr(e))

In [4]:
pkt = get_bgp_update_packet()
p_nlri = get_bgp_update_with_2_nlri()
p_hijack = get_bgp_update_151_hijack_packet()

FileNotFoundError: [Errno 2] No such file or directory: '/home/greg/BGP_Chain/Seed_scalable_complete/bgp_smart_contracts/pcaps/2022-10-17-14:37:04/as150.pcap'

In [553]:
p_hijack.show()

###[ cooked linux ]### 
  pkttype   = sent-by-us
  lladdrtype= 0x1
  lladdrlen = 6
  src       = '\x02B\nd\x00\\x97'
  proto     = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0xc0
     len       = 119
     id        = 11230
     flags     = DF
     frag      = 0
     ttl       = 1
     proto     = tcp
     chksum    = 0x3721
     src       = 10.100.0.151
     dst       = 10.100.0.100
     \options   \
###[ TCP ]### 
        sport     = bgp
        dport     = 39055
        seq       = 2321666824
        ack       = 2277178902
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 509
        chksum    = 0x8d0
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (1664964607, 674672228))]
###[ HEADER ]### 
           marker    = 0xffffffffffffffffffffffffffffffff
           len       = 67
           type      = UPDATE
###[ UPDATE ]### 
              withdrawn_routes_len= 0
              \withd

In [61]:
pkt[BGPUpdate].path_attr[1]

<BGPPathAttr  type_flags=Transitive type_code=AS_PATH attr_len=6 attribute=<BGPPAASPath  segments=[<ASPathSegment  segment_type=AS_SEQUENCE segment_length=1 segment_value=[0] |>, <ASPathSegment  segment_type=0 segment_length=150 |>] |> |>

In [62]:
pkt[BGPUpdate].path_attr[1].attribute.segments[0]

<ASPathSegment  segment_type=AS_SEQUENCE segment_length=1 segment_value=[0] |>

In [9]:
pkt[BGPUpdate].path_attr[1].attribute.segments[0].segment_length

1

In [184]:
pkt[BGPUpdate].nlri

[<BGPNLRI_IPv4  prefix=10.150.0.0/24 |>]

In [148]:
for nlri in pkt[BGPUpdate].nlri:
    print(nlri.show())

###[ IPv4 NLRI ]### 
  prefix    = 10.150.0.0/24

None


In [681]:
nlri = pkt[BGPUpdate].nlri[0]
print(bytes(nlri))
nlri.show()

nlri_hijack = p_hijack[BGPUpdate].nlri[0]
print('nlri to remove: ' + str(bytes(nlri_hijack)))
nlri_hijack_bytes = bytes(nlri_hijack)

b'\x18\n\x96\x00'
###[ IPv4 NLRI ]### 
  prefix    = 10.150.0.0/24

nlri to remove: b'\x19\n\x98\x00\x00'


In [197]:
type(nlri)

scapy.contrib.bgp.BGPNLRI_IPv4

In [199]:
print(nlri)
print(nlri.show())

b'\x18\n\x96\x00'
###[ IPv4 NLRI ]### 
  prefix    = 10.150.0.0/24

None


In [538]:
# new_nlri = BGPFieldIPv4("prefix", "10.150.1.1/25")
new_nlri = BGPNLRI_IPv4(prefix="10.150.1.0/24")
print(new_nlri)
type(new_nlri)
new_nlri_bytes = bytes(new_nlri)
new_nlri_raw = raw(new_nlri)
print(new_nlri_bytes)
print(new_nlri_raw)
# print(new_nlri.prefix)

b'\x18\n\x96\x01'
b'\x18\n\x96\x01'
b'\x18\n\x96\x01'


In [325]:
len(pkt)

130

In [378]:
del  pkt[BGPUpdate].nlri[0]
# if len(eval(cmd).nlri[num]) == 0:
#   eval(cmd).nlri[0].prefix.append('0.0.0.0/0')

In [372]:
pkt[BGPUpdate].nlri

[scapy.packet.Raw]

In [382]:
pkt[BGPUpdate].nlri.append(new_nlri)
# pkt[BGPUpdate].nlri.append(new_nlri_bytes)


In [370]:
new_nlri = new_nlri.default_payload_class(pkt[BGPUpdate])

In [371]:
pkt[BGPUpdate].nlri[0] = new_nlri

In [255]:
print(bytes(pkt[BGPUpdate].nlri[0]))
print(bytes(pkt[BGPUpdate].nlri[1]))


b'\x18\n\x96\x00'
b'\x18\n\x96\x01'


In [353]:
del pkt[IP].chksum
del pkt[TCP].chksum
del pkt[IP].len

In [373]:
# pkt[BGPUpdate].nlri.append(new_nlri)
pkt[BGPUpdate].nlri[0].show2()
pkt[BGPUpdate].show2()

TypeError: Packet.show2() missing 1 required positional argument: 'self'

In [536]:
p_nlri.show2()

###[ cooked linux ]### 
  pkttype   = unicast
  lladdrtype= 0x1
  lladdrlen = 6
  src       = '\x02B\nh\x00\x0c'
  proto     = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0xc0
     len       = 641
     id        = 4998
     flags     = DF
     frag      = 0
     ttl       = 1
     proto     = tcp
     chksum    = 0x4db2
     src       = 10.104.0.12
     dst       = 10.104.0.164
     \options   \
###[ TCP ]### 
        sport     = bgp
        dport     = 59473
        seq       = 2063855751
        ack       = 61297140
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 510
        chksum    = 0x17f3
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (3262136288, 1326917326))]
###[ HEADER ]### 
           marker    = 0xffffffffffffffffffffffffffffffff
           len       = 152
           type      = UPDATE
###[ UPDATE ]### 
              withdrawn_routes_len= 0
              \withdrawn_

In [535]:
pkt.show2()

###[ cooked linux ]### 
  pkttype   = sent-by-us
  lladdrtype= 0x1
  lladdrlen = 6
  src       = '\x02B\nd\x00\\x96'
  proto     = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0xc0
     len       = 114
     id        = 16538
     flags     = DF
     frag      = 0
     ttl       = 1
     proto     = tcp
     chksum    = 0x226b
     src       = 10.100.0.150
     dst       = 10.100.0.100
     \options   \
###[ TCP ]### 
        sport     = 55713
        dport     = bgp
        seq       = 548780263
        ack       = 1635794624
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 502
        chksum    = 0x1626
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (621282715, 795365190))]
###[ HEADER ]### 
           marker    = 0xffffffffffffffffffffffffffffffff
           len       = 62
           type      = UPDATE
###[ UPDATE ]### 
              withdrawn_routes_len= 0
              \withdr

In [633]:
# del pkt[IP].chksum
# del pkt[TCP].chksum
# print(bytes(pkt))
p_hijack = get_bgp_update_151_hijack_packet()
p_hijack.show()
print(bytes(p_hijack))
p_hijack_bytes = bytearray(bytes(p_hijack))
print()
print(p_hijack_bytes)
print(len(p_hijack_bytes))
print(p_hijack_bytes[126:131])
del p_hijack_bytes[126:131]
print(p_hijack_bytes)
p_hijack_reconstructed = CookedLinux(bytes(p_hijack_bytes))
p_hijack_reconstructed[BGPHeader].len = p_hijack_reconstructed[BGPHeader].len - 5
p_hijack_reconstructed.show2()

###[ cooked linux ]### 
  pkttype   = sent-by-us
  lladdrtype= 0x1
  lladdrlen = 6
  src       = '\x02B\nd\x00\\x97'
  proto     = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0xc0
     len       = 119
     id        = 11230
     flags     = DF
     frag      = 0
     ttl       = 1
     proto     = tcp
     chksum    = 0x3721
     src       = 10.100.0.151
     dst       = 10.100.0.100
     \options   \
###[ TCP ]### 
        sport     = bgp
        dport     = 39055
        seq       = 2321666824
        ack       = 2277178902
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 509
        chksum    = 0x8d0
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (1664964607, 674672228))]
###[ HEADER ]### 
           marker    = 0xffffffffffffffffffffffffffffffff
           len       = 67
           type      = UPDATE
###[ UPDATE ]### 
              withdrawn_routes_len= 0
              \withd

In [650]:
p_hijack = get_bgp_update_151_hijack_packet()
p_hijack.show2()
print(bytes(p_hijack))
print(len(p_hijack))
nlri_hijack = p_hijack[BGPUpdate].nlri[0]
print('nlri to remove: ' + str(bytes(nlri_hijack)))
nlri_hijack_bytes = bytes(nlri_hijack)
nlri_hijack_bytes_list = []
nlri_hijack_bytes_list.append(nlri_hijack_bytes)
print(len(nlri_hijack_bytes_list))
p_hijack_bytes = bytearray(bytes(p_hijack))

if nlri_hijack_bytes in p_hijack_bytes:
    print("suhhhh")
#     p_hijack_bytes.remove(nlri_hijack_bytes)
    index = p_hijack_bytes.index(nlri_hijack_bytes)
    print(index)
#     p_hijack_reconstructed = filter(lambda i: i not in nlri_hijack_bytes_list, p_hijack_bytes)
#     p_hijack_reconstructed = CookedLinux(bytes(p_hijack_bytes))
#     print(len(p_hijack_reconstructed))
#     p_hijack_reconstructed[BGPHeader].len = p_hijack_reconstructed[BGPHeader].len - 5
#     del pkt_reconstructed[IP].chksum
#     del pkt_reconstructed[TCP].chksum 
#     pkt_reconstructed[IP].len = pkt_reconstructed[IP].len - 5
#     pkt_reconstructed.show2()

###[ cooked linux ]### 
  pkttype   = sent-by-us
  lladdrtype= 0x1
  lladdrlen = 6
  src       = '\x02B\nd\x00\\x97'
  proto     = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0xc0
     len       = 119
     id        = 11230
     flags     = DF
     frag      = 0
     ttl       = 1
     proto     = tcp
     chksum    = 0x3721
     src       = 10.100.0.151
     dst       = 10.100.0.100
     \options   \
###[ TCP ]### 
        sport     = bgp
        dport     = 39055
        seq       = 2321666824
        ack       = 2277178902
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 509
        chksum    = 0x8d0
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (1664964607, 674672228))]
###[ HEADER ]### 
           marker    = 0xffffffffffffffffffffffffffffffff
           len       = 67
           type      = UPDATE
###[ UPDATE ]### 
              withdrawn_routes_len= 0
              \withd

In [690]:
p_hijack_bytes = b'E\xc0\x00w\xec\xd0@\x00\x01\x06v.\nd\x00\x97\nd\x00d\x00\xb3\x89\x1fj\x19+\xc4\x93\xc9\xa4\xe8\x80\x18\x01\xfd\xb3\xbb\x00\x00\x01\x01\x08\nc\x8f\x94F(\x88\xea\xa9\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00C\x02\x00\x00\x00#@\x01\x01\x00@\x02\x06\x02\x01\x00\x00\x00\x97@\x03\x04\nd\x00\x97\xc0 \x0c\x00\x00\x00\x97\x00\x00\x00\x00\x00\x00\x00\x00\x19\n\x98\x00\x00\x18\n\x97\x00'
print(p_hijack_bytes)
p_hijack = IP(p_hijack_bytes)
p_hijack.show2()
p_hijack_bytes = bytearray(p_hijack_bytes)

nlri_hijack = p_hijack[BGPUpdate].nlri[0]
print('nlri to remove: ' + str(bytes(nlri_hijack)))
# print('nlri to remove show: ' + nlri_hijack.show2())
nlri_hijack_bytes = bytes(nlri_hijack)
len_nlri_hijack_bytes = len(nlri_hijack_bytes)
print(len_nlri_hijack_bytes)

try:
    index = p_hijack_bytes.index(nlri_hijack_bytes)
    print(index)

    del p_hijack_bytes[index:index+len_nlri_hijack_bytes]
    pkt_reconstructed = IP(bytes(p_hijack_bytes))
    print("modify length of bgp packet")
    pkt_reconstructed[BGPHeader].len = pkt_reconstructed[BGPHeader].len - 5
    del pkt_reconstructed[IP].chksum
    del pkt_reconstructed[TCP].chksum 
    pkt_reconstructed[IP].len = pkt_reconstructed[IP].len - 5
    pkt_reconstructed.show2()
except ValueError as v:
    print("Error. nlri not found in packet. this is weird: " + repr(v))
    print("nlri not found:")
    nlri.show()
#     print("nlri that is not found: " + str(nlri_hijack.show()))


b'E\xc0\x00w\xec\xd0@\x00\x01\x06v.\nd\x00\x97\nd\x00d\x00\xb3\x89\x1fj\x19+\xc4\x93\xc9\xa4\xe8\x80\x18\x01\xfd\xb3\xbb\x00\x00\x01\x01\x08\nc\x8f\x94F(\x88\xea\xa9\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00C\x02\x00\x00\x00#@\x01\x01\x00@\x02\x06\x02\x01\x00\x00\x00\x97@\x03\x04\nd\x00\x97\xc0 \x0c\x00\x00\x00\x97\x00\x00\x00\x00\x00\x00\x00\x00\x19\n\x98\x00\x00\x18\n\x97\x00'
###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0xc0
  len       = 119
  id        = 60624
  flags     = DF
  frag      = 0
  ttl       = 1
  proto     = tcp
  chksum    = 0x762e
  src       = 10.100.0.151
  dst       = 10.100.0.100
  \options   \
###[ TCP ]### 
     sport     = bgp
     dport     = 35103
     seq       = 1780034500
     ack       = 2479465704
     dataofs   = 8
     reserved  = 0
     flags     = PA
     window    = 509
     chksum    = 0xb3bb
     urgptr    = 0
     options   = [('NOP', None), ('NOP', None), ('Timestamp', (1670353990, 680061609))]
###[ HE

In [793]:
pkt_bytes = b'E\xc0\x00w\xec\xd0@\x00\x01\x06v.\nd\x00\x97\nd\x00d\x00\xb3\x89\x1fj\x19+\xc4\x93\xc9\xa4\xe8\x80\x18\x01\xfd\xb3\xbb\x00\x00\x01\x01\x08\nc\x8f\x94F(\x88\xea\xa9\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00C\x02\x00\x00\x00#@\x01\x01\x00@\x02\x06\x02\x01\x00\x00\x00\x97@\x03\x04\nd\x00\x97\xc0 \x0c\x00\x00\x00\x97\x00\x00\x00\x00\x00\x00\x00\x00\x19\n\x98\x00\x00\x18\n\x97\x00'
pkt = IP(pkt_bytes)
# pkt.show()
layer_index = 0
for payload in pkt.iterpayloads():
    if isinstance(payload,  scapy.contrib.bgp.BGPHeader):
        most_recent_bgp_header = payload
    elif isinstance(payload, scapy.contrib.bgp.BGPUpdate):
        layer_index += 1
        print(type(payload))
        update = BGPUpdate_v2(most_recent_bgp_header, payload, layer_index)
        print(update)
        if not update.has_withdraw_routes() and update.has_nlri_advertisements():
            # origin_asn = get_update_origin_asn(update)
            for count, nlri in enumerate(update.nlri()):
                segment = update.get_segment(nlri)
                print("nlri count: " + str(count))
                print("BGP NLRI check: " + str(nlri.prefix))
                print ("Advertised Segment: " + str(segment))
                print ("validating advertisement for ASN: " + str(update.get_origin_asn()))
                
                # test
                if segment[2] == '25':
                    print('suhh')
                    handle_invalid_advertisement(pkt, nlri, update)
                    break


<class 'scapy.contrib.bgp.BGPUpdate'>
reverting to segment length
nlri count: 0
BGP NLRI check: 10.152.0.0/25
Advertised Segment: [151, '10.152.0.0', '25']
validating advertisement for ASN: 151
suhh
###[ UPDATE ]### 
  withdrawn_routes_len= 0
  \withdrawn_routes\
  path_attr_len= 35
  \path_attr \
   |###[ BGPPathAttr ]### 
   |  type_flags= Transitive
   |  type_code = ORIGIN
   |  attr_len  = 1
   |  \attribute \
   |   |###[ ORIGIN ]### 
   |   |  origin    = IGP
   |###[ BGPPathAttr ]### 
   |  type_flags= Transitive
   |  type_code = AS_PATH
   |  attr_len  = 6
   |  \attribute \
   |   |###[ AS_PATH (RFC 4271) ]### 
   |   |  \segments  \
   |   |   |###[ ASPathSegment ]### 
   |   |   |  segment_type= AS_SEQUENCE
   |   |   |  segment_length= 1
   |   |   |  segment_value= [0]
   |   |   |###[ ASPathSegment ]### 
   |   |   |  segment_type= 0
   |   |   |  segment_length= 151
   |   |   |  segment_value= []
   |###[ BGPPathAttr ]### 
   |  type_flags= Transitive
   |  type_code 

In [792]:
def handle_invalid_advertisement(pkt, nlri, update):
#     print(pkt.getlayer(scapy.contrib.bgp.BGPHeader))
#     pkt.show()
#     bgp_header = pkt.getlayer(scapy.contrib.bgp.BGPHeader, update.get_layer_index())
    bgp_update = pkt.getlayer(scapy.contrib.bgp.BGPUpdate, update.get_layer_index())
    
    bgp_update.show()
    nlri_bytes = bytes(nlri)
    print("len of nlri: " + str(len(nlri_bytes)))
    # pkt_byte_array = bytearray(bytes(self.pkt))
    update_byte_array = bytearray(bytes(bgp_update))
    print(update_byte_array)
        
# #     try: 
    index = update_byte_array.index(nlri_bytes)
    print("start index of nlri to remove: " + str(index))

    # delete the nlri from the packet
    del update_byte_array[index:index+len(nlri_bytes)]

    new_bgp_update = scapy.contrib.bgp.BGPUpdate(bytes(update_byte_array))
#     new_bgp_update.show()
#     bgp_header.len = bgp_header.len - len(nlri_bytes)
    
    pkt_bytes = bytearray(bytes(pkt))

    
    pkt_index = pkt_bytes.index(bytes(bgp_update))
    print(pkt_index)
    
    del pkt_bytes[pkt_index:pkt_index+len(bytes(bgp_update))]
    pkt_bytes[pkt_index:pkt_index] = update_byte_array
    
    new_pkt = IP(bytes(pkt_bytes))
    
    bgp_header = new_pkt.getlayer(scapy.contrib.bgp.BGPHeader, update.get_layer_index())
    bgp_header.len = bgp_header.len - len(nlri_bytes)
    del new_pkt[IP].chksum
    del new_pkt[TCP].chksum
    new_pkt[IP].len = new_pkt[IP].len - len(nlri_bytes)
    new_pkt.show2()
    
    
    
    
    

#         self.pkt = IP(bytes(pkt_byte_array))
#         print("modify length of bgp packet")
#         self.pkt[BGPHeader].len = self.pkt[BGPHeader].len - len(nlri_bytes)
#         del self.pkt[IP].chksum
#         del self.pkt[TCP].chksum 
#         self.pkt[IP].len = self.pkt[IP].len - len(nlri_bytes)
#         print("modified packet: ") 
#         self.pkt.show2()
#         self.set_modified()
#     except ValueError as v:
#         print("Error. nlri not found in packet. this is weird: " + repr(v))
#         print("nlri not found:")
#         nlri.show()    

In [761]:
class BGPUpdate_v2:
    def __init__(self, bgp_header, bgp_update, layer_index):
        self.bgp_header = bgp_header
        self.bgp_update = bgp_update
        self.layer_index = layer_index
        self.origin_asn = -1

    def get_layer_index(self):
        return self.layer_index

    def get_origin_asn(self):
        return self.origin_asn
    
    def has_nlri_advertisements(self):
        if self.bgp_update.path_attr_len > 0:
            return True
        return False
    
    def has_withdraw_routes(self):
        if self.bgp_update.withdrawn_routes_len > 0:
            return True
        return False

    
    def get_origin_asn_from_payload(self):
        if len(self.bgp_update.path_attr[1].attribute.segments[-1].segment_value) > 1: #grabs last asn in as_path (origin)
            self.origin_asn = self.bgp_update.path_attr[1].attribute.segments[-1].segment_value[-1]
        else:
            print('reverting to segment length')
            self.origin_asn = self.bgp_update.path_attr[1].attribute.segments[-1].segment_length
        return self.origin_asn

    def nlri(self):
        return self.bgp_update.nlri
   
    def get_segment(self, nlri):
        if self.origin_asn == -1:
            self.get_origin_asn_from_payload()
        return [self.origin_asn, str(nlri.prefix).split('/')[0], str(nlri.prefix).split('/')[1]]

In [427]:
print(raw(pkt))

b'\x00\x04\x00\x01\x00\x06\x02B\nd\x00\x96\x00\x00\x08\x00E\xc0\x00r@\x9a@\x00\x01\x06"k\nd\x00\x96\nd\x00d\xd9\xa1\x00\xb3 \xb5\xb8\xe7a\x80>\xc0\x80\x18\x01\xf6\x16&\x00\x00\x01\x01\x08\n%\x08\x05\x9b/hOF\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00>\x02\x00\x00\x00#@\x01\x01\x00@\x02\x06\x02\x01\x00\x00\x00\x96@\x03\x04\nd\x00\x96\xc0 \x0c\x00\x00\x00\x96\x00\x00\x00\x00\x00\x00\x00\x00\x18\n\x96\x00'


In [544]:
b_pkt = bytearray(bytes(pkt))
b_pkt
print(len(b_pkt))
print(hex(b_pkt[129]))
print(type(hex(b_pkt[129])))

130
0x0
<class 'str'>


In [545]:
# new_val = 0
# b_pkt[129] = new_val
print(b_pkt)
print(new_nlri_bytes)
b_pkt[126:130] = new_nlri_bytes
print(b_pkt)

bytearray(b'\x00\x04\x00\x01\x00\x06\x02B\nd\x00\x96\x00\x00\x08\x00E\xc0\x00r@\x9a@\x00\x01\x06"k\nd\x00\x96\nd\x00d\xd9\xa1\x00\xb3 \xb5\xb8\xe7a\x80>\xc0\x80\x18\x01\xf6\x16&\x00\x00\x01\x01\x08\n%\x08\x05\x9b/hOF\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00>\x02\x00\x00\x00#@\x01\x01\x00@\x02\x06\x02\x01\x00\x00\x00\x96@\x03\x04\nd\x00\x96\xc0 \x0c\x00\x00\x00\x96\x00\x00\x00\x00\x00\x00\x00\x00\x18\n\x96\x00')
b'\x18\n\x96\x01'
bytearray(b'\x00\x04\x00\x01\x00\x06\x02B\nd\x00\x96\x00\x00\x08\x00E\xc0\x00r@\x9a@\x00\x01\x06"k\nd\x00\x96\nd\x00d\xd9\xa1\x00\xb3 \xb5\xb8\xe7a\x80>\xc0\x80\x18\x01\xf6\x16&\x00\x00\x01\x01\x08\n%\x08\x05\x9b/hOF\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00>\x02\x00\x00\x00#@\x01\x01\x00@\x02\x06\x02\x01\x00\x00\x00\x96@\x03\x04\nd\x00\x96\xc0 \x0c\x00\x00\x00\x96\x00\x00\x00\x00\x00\x00\x00\x00\x18\n\x96\x01')


In [513]:
# b_pkt.extend(b'\x00')
b_pkt.extend(new_nlri_bytes)

In [514]:
print(b_pkt)

bytearray(b'\x00\x04\x00\x01\x00\x06\x02B\nd\x00\x96\x00\x00\x08\x00E\xc0\x00r@\x9a@\x00\x01\x06"k\nd\x00\x96\nd\x00d\xd9\xa1\x00\xb3 \xb5\xb8\xe7a\x80>\xc0\x80\x18\x01\xf6\x16&\x00\x00\x01\x01\x08\n%\x08\x05\x9b/hOF\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00>\x02\x00\x00\x00#@\x01\x01\x00@\x02\x06\x02\x01\x00\x00\x00\x96@\x03\x04\nd\x00\x96\xc0 \x0c\x00\x00\x00\x96\x00\x00\x00\x00\x00\x00\x00\x00\x18\n\x96\x01\x00\x18\n\x96\x01')


In [546]:
# pkt_reconstructed = CookedLinux(b_pkt)
pkt_reconstructed = CookedLinux(bytes(b_pkt))
del pkt_reconstructed[IP].chksum 

In [547]:
print(pkt_reconstructed.show2())

###[ cooked linux ]### 
  pkttype   = sent-by-us
  lladdrtype= 0x1
  lladdrlen = 6
  src       = '\x02B\nd\x00\\x96'
  proto     = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0xc0
     len       = 114
     id        = 16538
     flags     = DF
     frag      = 0
     ttl       = 1
     proto     = tcp
     chksum    = 0x226b
     src       = 10.100.0.150
     dst       = 10.100.0.100
     \options   \
###[ TCP ]### 
        sport     = 55713
        dport     = bgp
        seq       = 548780263
        ack       = 1635794624
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 502
        chksum    = 0x1626
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (621282715, 795365190))]
###[ HEADER ]### 
           marker    = 0xffffffffffffffffffffffffffffffff
           len       = 62
           type      = UPDATE
###[ UPDATE ]### 
              withdrawn_routes_len= 0
              \withdr

In [523]:
pkt_reconstructed[BGPUpdate].nlri.append(new_nlri)

In [78]:
print(bytes(pkt))

b'\x00\x04\x00\x01\x00\x06\x02B\nd\x00\x96\x00\x00\x08\x00E\xc0\x00r@\x9a@\x00\x01\x06"k\nd\x00\x96\nd\x00d\xd9\xa1\x00\xb3 \xb5\xb8\xe7a\x80>\xc0\x80\x18\x01\xf6(e\x00\x00\x01\x01\x08\n%\x08\x05\x9b/hOF\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00>\x02\x00\x00\x00#@\x01\x01\x00@\x02\x06\x02\x01\x00\x00\x00\x96@\x03\x04\nd\x00\x96\xc0 \x0c\x00\x00\x00\x96\x00\x00\x00\x00\x00\x00\x00\x00\x18\n\x96\x00'


In [40]:
# val = pkt[BGPUpdate].show()
val = pkt[BGPUpdate].path_attr[3][Raw].load
pkt[BGPUpdate].path_attr[3][Raw].load = val.decode('utf-8')
print(type(val))

print(pkt[BGPUpdate].path_attr[3][Raw].load)

<class 'bytes'>
b'b"b\'\\\\x00\\\\x00\\\\x00\\\\x96\\\\x00\\\\x00\\\\x00\\\\x00\\\\x00\\\\x00\\\\x00\\\\x00\'"'


In [41]:
val = b'\x00\x00\x00\x97\x00\x00\x00\x00\x00\x00\x00\x00'

In [42]:
str(val)

"b'\\x00\\x00\\x00\\x97\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00'"

In [43]:
print(val.decode('utf-8'))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x97 in position 3: invalid start byte

In [61]:
pkt[BGPUpdate].nlri[0].prefix = "10.150.0.0/24"

In [68]:
pkt[BGPUpdate].nlri[0]

<BGPNLRI_IPv4  prefix=10.150.0.0/24 |>

In [80]:
pkt[BGPUpdate].nlri = []

In [86]:
pkt[BGPUpdate].show()

###[ UPDATE ]### 
  withdrawn_routes_len= 0
  \withdrawn_routes\
  path_attr_len= 35
  \path_attr \
   |###[ BGPPathAttr ]### 
   |  type_flags= Transitive
   |  type_code = ORIGIN
   |  attr_len  = 1
   |  \attribute \
   |   |###[ ORIGIN ]### 
   |   |  origin    = IGP
   |###[ BGPPathAttr ]### 
   |  type_flags= Transitive
   |  type_code = AS_PATH
   |  attr_len  = 6
   |  \attribute \
   |   |###[ AS_PATH (RFC 4271) ]### 
   |   |  \segments  \
   |   |   |###[ ASPathSegment ]### 
   |   |   |  segment_type= AS_SEQUENCE
   |   |   |  segment_length= 1
   |   |   |  segment_value= [0]
   |   |   |###[ ASPathSegment ]### 
   |   |   |  segment_type= 0
   |   |   |  segment_length= 150
   |   |   |  segment_value= []
   |###[ BGPPathAttr ]### 
   |  type_flags= Transitive
   |  type_code = NEXT_HOP
   |  attr_len  = 4
   |  \attribute \
   |   |###[ NEXT_HOP ]### 
   |   |  next_hop  = 10.100.0.150
   |###[ BGPPathAttr ]### 
   |  type_flags= Transitive+Optional
   |  type_code = 32


In [204]:
bgp_update = pkt[BGPUpdate]

In [206]:
pkt.show2()

###[ cooked linux ]### 
  pkttype   = sent-by-us
  lladdrtype= 0x1
  lladdrlen = 6
  src       = '\x02B\nd\x00\\x96'
  proto     = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0xc0
     len       = 114
     id        = 16538
     flags     = DF
     frag      = 0
     ttl       = 1
     proto     = tcp
     chksum    = 0x226b
     src       = 10.100.0.150
     dst       = 10.100.0.100
     \options   \
###[ TCP ]### 
        sport     = 55713
        dport     = bgp
        seq       = 548780263
        ack       = 1635794624
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 502
        chksum    = 0x1626
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (621282715, 795365190))]
###[ HEADER ]### 
           marker    = 0xffffffffffffffffffffffffffffffff
           len       = 62
           type      = UPDATE
###[ UPDATE ]### 
              withdrawn_routes_len= 0
              \withdr

In [198]:
pkt[TCP].remove_payload()

In [176]:
# del bgp_update.nlri
bgp_update.nlri = []

In [199]:
del pkt[IP].chksum

In [178]:
pkt.show2()

###[ cooked linux ]### 
  pkttype   = sent-by-us
  lladdrtype= 0x1
  lladdrlen = 6
  src       = '\x02B\nd\x00\\x96'
  proto     = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0xc0
     len       = 114
     id        = 16538
     flags     = DF
     frag      = 0
     ttl       = 1
     proto     = tcp
     chksum    = 0x226b
     src       = 10.100.0.150
     dst       = 10.100.0.100
     \options   \
###[ TCP ]### 
        sport     = 55713
        dport     = bgp
        seq       = 548780263
        ack       = 1635794624
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 502
        chksum    = 0x1626
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (621282715, 795365190))]



In [200]:
pkt = pkt/bgp_update

In [202]:
pkt.show2()

###[ cooked linux ]### 
  pkttype   = sent-by-us
  lladdrtype= 0x1
  lladdrlen = 6
  src       = '\x02B\nd\x00\\x96'
  proto     = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0xc0
     len       = 114
     id        = 16538
     flags     = DF
     frag      = 0
     ttl       = 1
     proto     = tcp
     chksum    = 0x226b
     src       = 10.100.0.150
     dst       = 10.100.0.100
     \options   \
###[ TCP ]### 
        sport     = 55713
        dport     = bgp
        seq       = 548780263
        ack       = 1635794624
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 502
        chksum    = 0x1626
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (621282715, 795365190))]
###[ Raw ]### 
           load      = '\x00\x00\x00#@\x01\x01\x00@\x02\x06\x02\x01\x00\x00\x00\\x96@\x03\x04\nd\x00\\x96\\xc0 \x0c\x00\x00\x00\\x96\x00\x00\x00\x00\x00\x00\x00\x00\x18\n\\x96\x00'

